# ADK CoLab Boilerplate

Last Update:

- June 7, 2025
- ADK == 1.1.1

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Artifacts_service.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FArtifacts_service.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Artifacts_service.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/main/Artifacts_service.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Environment Setup

In [35]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}
ARTIFACTS_BUCKET = 'hupili-genai-bb-public' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [36]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [37]:
from google.colab import auth
auth.authenticate_user()

In [38]:
!pip install deprecated
!pip install google-adk==1.1.1

In [39]:
from google import adk

In [40]:
import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()
    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()
        agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

        if is_final_response:
            print('>>> inside final response...')
            final_response = event.content.parts[0].text
            end_time = time.time()
            elapsed_time_ms = round((end_time - start_time) * 1000, 3)
            print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-----------------------------")
        elif function_calls:
            print('+++ inside the function call...')
            for function_call in function_calls:
                print(f"function, [args]:  {function_call.name}, {function_call.args}")
        elif function_responses:
            print('--- inside the function call response...')
            # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
            # if not event.actions.pending:
            if True:
                for function_response in function_responses:
                    details = function_response.response
                    recommended_list = list(details.values())
                    print(f"Function Name: {function_response.name}")
                    result=json.dumps(recommended_list)
                    print(f"Function Results {result}")
            else:
                print(agent_res)
    print(f"Total elapsed time: {elapsed_time_ms}")


In [41]:
import random
import asyncio

from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

async def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  session_service = InMemorySessionService()
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = await session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service)
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

In [42]:
MODEL = "gemini-2.0-flash"
APP_NAME = "simple_search_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "simple_search_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

In [43]:
call_agent = await caller_factory(simple_search_agent)

In [44]:
events = call_agent('Hello')

# GCS Artifacts

In [45]:
from google.adk.artifacts import GcsArtifactService

# Specify the GCS bucket name
gcs_bucket_name_py = ARTIFACTS_BUCKET # Replace with your bucket name

try:
    gcs_service_py = GcsArtifactService(bucket_name=gcs_bucket_name_py)
    print(f"Python GcsArtifactService initialized for bucket: {gcs_bucket_name_py}")
    # Ensure your environment has credentials to access this bucket.
    # e.g., via Application Default Credentials (ADC)

    # Then pass it to the Runner
    # runner = Runner(..., artifact_service=gcs_service_py)

except Exception as e:
    # Catch potential errors during GCS client initialization (e.g., auth issues)
    print(f"Error initializing Python GcsArtifactService: {e}")
    # Handle the error appropriately - maybe fall back to InMemory or raise

async def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  session_service = InMemorySessionService()
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = await session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service, artifact_service=gcs_service_py)
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

Python GcsArtifactService initialized for bucket: hupili-genai-bb-public


In [91]:
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types
from google.adk.agents.callback_context import CallbackContext # Or ToolContext
import base64
from google.cloud import storage
import mimetypes # Import mimetypes

async def save_text_file(
    filename: str, file_text: str, mime_type: str, tool_context: ToolContext
) -> str:
    """Save a text file to artifacts service.
    """
    try:
        file_bytes = file_text.encode('utf-8')
        artifact = types.Part(inline_data=types.Blob(data=file_bytes, mime_type=mime_type))
        # Pass only the filename to save_artifact
        await tool_context.save_artifact(filename=filename, artifact=artifact)
        return filename
    except Exception as e:
        print(f"Error saving file {e}")
        return ""

async def load_text_file(
    filename: str, tool_context: ToolContext
) -> str:
    """Load a text file from artifacts service.
    """
    try:
        artifact = await tool_context.load_artifact(filename=filename)
        file_bytes = artifact.inline_data.data
        file_text = file_bytes.decode('utf-8')
        return file_text
    except Exception as e:
        print(f"Error loading file {e}")
        return ""


async def list_files(tool_context: ToolContext) -> str:
    """Tool to list available artifacts for the user."""
    try:
        available_files = await tool_context.list_artifacts()
        if not available_files:
            return "You have no saved artifacts."
        else:
            # Format the list for the user/LLM
            file_list_str = "\n".join([f"- {fname}" for fname in available_files])
            return f"Here are your available Python artifacts:\n{file_list_str}"
    except ValueError as e:
        print(f"Error listing Python artifacts: {e}. Is ArtifactService configured?")
        return "Error: Could not list Python artifacts."
    except Exception as e:
        print(f"An unexpected error occurred during Python artifact list: {e}")
        return "Error: An unexpected error occurred while listing Python artifacts."

PUBLISH_BUCKET = ARTIFACTS_BUCKET

async def publish_file(file_name: str, path_prefix: str, tool_context: ToolContext) -> str:
    """Tool to publish a file from the artifact service to public bucket.

    Args:
    - file_name: name of the file to publish
    - path_prefix: the path prefix on the public bucket
    - tool_context: ToolContext object
    """
    try:
        artifact = await tool_context.load_artifact(filename=file_name)
        file_bytes = artifact.inline_data.data
        file_text = file_bytes.decode('utf-8')
        print(f'loaded artifacts: {file_text[: 100]}')

        if path_prefix.startswith("/"):
          path_prefix = path_prefix[1:]
        if path_prefix.endswith("/"):
          path_prefix = path_prefix[:-1]

        # Derive content type from file extension
        mime_type, _ = mimetypes.guess_type(file_name)
        if mime_type is None:
            mime_type = 'application/octet-stream' # Default if type cannot be guessed

        # write the file to PUBLISH_BUCKET
        storage_client = storage.Client()
        bucket = storage_client.bucket(PUBLISH_BUCKET)
        blob = bucket.blob(f"{path_prefix}/{file_name}")

        blob.upload_from_string(file_bytes.decode('utf-8'), content_type=mime_type)

        gs_url = f"gs://{PUBLISH_BUCKET}/{path_prefix}/{file_name}"
        # Add a nounce to invalidate the cache on published URL
        nounce = str(random.randint(0,10000))
        public_url = f'https://storage.googleapis.com/{PUBLISH_BUCKET}/{path_prefix}/{file_name}?nounce={nounce}'
        return f"File '{file_name}' published to {public_url}"
    except Exception as e:
        return f"Error publishing file: {e}"

In [95]:
MODEL = "gemini-2.0-flash"
APP_NAME = "artifact_agent"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "artifact_agent"

# Agent
artifact_agent = Agent(
    model=MODEL,
    name="artifact_agent",
    description="Agent to fulfill user query and save results to artifacts",
    instruction="I can answer your questions and save the results to artifacts",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[save_text_file, load_text_file, list_files, publish_file],
)

In [96]:
call_agent = await caller_factory(artifact_agent)

In [97]:
events = call_agent('write a plan in markdown format to conduct research on AI Agent 2025. Save it to plan.md')
pprint_events(events)

+++ inside the function call...
function, [args]:  save_text_file, {'mime_type': 'text/markdown', 'filename': 'plan.md', 'file_text': '# Research Plan: AI Agents in 2025\n\n## Goal\n\nTo understand the landscape of AI Agents in 2025, including their capabilities, applications, and potential impact.\n\n## Objectives\n\n*   Identify the key trends driving the development of AI Agents.\n*   Analyze the capabilities of state-of-the-art AI Agents.\n*   Explore the potential applications of AI Agents in various industries.\n*   Assess the potential societal and economic impact of AI Agents.\n*   Identify the ethical considerations surrounding AI Agents.\n\n## Research Questions\n\n*   What are the key technological advancements enabling the development of more sophisticated AI Agents?\n*   What are the limitations of current AI Agents?\n*   In which industries are AI Agents expected to have the greatest impact?\n*   What are the potential risks and benefits of widespread adoption of AI Agent

In [98]:
# list(events)

In [99]:
events = call_agent('what are my files?')
pprint_events(events)

+++ inside the function call...
function, [args]:  list_files, {}
--- inside the function call response...
Function Name: list_files
Function Results ["Here are your available Python artifacts:\n- plan.md"]
>>> inside final response...
Final Response (1520.092 ms):
Your file is plan.md

-----------------------------
Total elapsed time: 1520.092


In [100]:
events = call_agent('Turn the plan into a beautiful webpage')
pprint_events(events)

>>> inside final response...
Final Response (1112.444 ms):
I can save the plan as an HTML file, but I can't create a "beautiful" webpage without more specific instructions on the desired look and feel. I will use a basic HTML structure and include the markdown content.

What filename do you want to use?

-----------------------------
Total elapsed time: 1112.444


In [101]:
events = call_agent('sure proceed with a basic web page and save to index.html')
pprint_events(events)

+++ inside the function call...
function, [args]:  load_text_file, {'filename': 'plan.md'}
--- inside the function call response...
Function Name: load_text_file
Function Results ["# Research Plan: AI Agents in 2025\n\n## Goal\n\nTo understand the landscape of AI Agents in 2025, including their capabilities, applications, and potential impact.\n\n## Objectives\n\n*   Identify the key trends driving the development of AI Agents.\n*   Analyze the capabilities of state-of-the-art AI Agents.\n*   Explore the potential applications of AI Agents in various industries.\n*   Assess the potential societal and economic impact of AI Agents.\n*   Identify the ethical considerations surrounding AI Agents.\n\n## Research Questions\n\n*   What are the key technological advancements enabling the development of more sophisticated AI Agents?\n*   What are the limitations of current AI Agents?\n*   In which industries are AI Agents expected to have the greatest impact?\n*   What are the potential risks a

+++ inside the function call...
function, [args]:  save_text_file, {'mime_type': 'text/html', 'file_text': '<!DOCTYPE html>\n<html>\n<head>\n  <title>AI Agent 2025 Research Plan</title>\n</head>\n<body>\n  <h1>AI Agent 2025 Research Plan</h1>\n  <p><strong>Goal:</strong> To understand the landscape of AI Agents in 2025, including their capabilities, applications, and potential impact.</p>\n  <h2>Objectives</h2>\n  <ul>\n    <li>Identify the key trends driving the development of AI Agents.</li>\n    <li>Analyze the capabilities of state-of-the-art AI Agents.</li>\n    <li>Explore the potential applications of AI Agents in various industries.</li>\n    <li>Assess the potential societal and economic impact of AI Agents.</li>\n    <li>Identify the ethical considerations surrounding AI Agents.</li>\n  </ul>\n  <h2>Research Questions</h2>\n  <ul>\n    <li>What are the key technological advancements enabling the development of more sophisticated AI Agents?</li>\n    <li>What are the limitatio

In [102]:
events = call_agent('publish that to /adk-101/demo-artifact-service-plan/')
pprint_events(events)

+++ inside the function call...
function, [args]:  publish_file, {'file_name': 'index.html', 'path_prefix': '/adk-101/demo-artifact-service-plan/'}
loaded artifacts: <!DOCTYPE html>
<html>
<head>
  <title>AI Agent 2025 Research Plan</title>
</head>
<body>
  <h1>AI A
--- inside the function call response...
Function Name: publish_file
Function Results ["File 'index.html' published to https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index.html?nounce=4394"]
>>> inside final response...
Final Response (2614.427 ms):
I have published the file `index.html` to `https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index.html?nounce=4394`.

-----------------------------
Total elapsed time: 2614.427


In [103]:
events = call_agent('add tailwind css to the webpage and save to index2.html')
pprint_events(events)

+++ inside the function call...
function, [args]:  load_text_file, {'filename': 'index.html'}
--- inside the function call response...
Function Name: load_text_file
Function Results ["<!DOCTYPE html>\n<html>\n<head>\n  <title>AI Agent 2025 Research Plan</title>\n</head>\n<body>\n  <h1>AI Agent 2025 Research Plan</h1>\n  <p><strong>Goal:</strong> To understand the landscape of AI Agents in 2025, including their capabilities, applications, and potential impact.</p>\n  <h2>Objectives</h2>\n  <ul>\n    <li>Identify the key trends driving the development of AI Agents.</li>\n    <li>Analyze the capabilities of state-of-the-art AI Agents.</li>\n    <li>Explore the potential applications of AI Agents in various industries.</li>\n    <li>Assess the potential societal and economic impact of AI Agents.</li>\n    <li>Identify the ethical considerations surrounding AI Agents.</li>\n  </ul>\n  <h2>Research Questions</h2>\n  <ul>\n    <li>What are the key technological advancements enabling the devel

+++ inside the function call...
function, [args]:  save_text_file, {'filename': 'index2.html', 'file_text': '<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>AI Agent 2025 Research Plan</title>\n    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">\n</head>\n<body class="bg-gray-100 font-sans">\n    <div class="container mx-auto p-8">\n        <h1 class="text-3xl font-bold mb-4">AI Agent 2025 Research Plan</h1>\n        <p class="mb-4"><strong class="font-medium">Goal:</strong> To understand the landscape of AI Agents in 2025, including their capabilities, applications, and potential impact.</p>\n\n        <h2 class="text-2xl font-semibold mt-6 mb-2">Objectives</h2>\n        <ul class="list-disc pl-5 mb-4">\n            <li>Identify the key trends driving the development of AI Agents.</li>\n            <li>Analyze the capabiliti

In [104]:
events = call_agent('publish that to /adk-101/demo-artifact-service-plan/')
pprint_events(events)

+++ inside the function call...
function, [args]:  publish_file, {'file_name': 'index2.html', 'path_prefix': '/adk-101/demo-artifact-service-plan/'}
loaded artifacts: <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content
--- inside the function call response...
Function Name: publish_file
Function Results ["File 'index2.html' published to https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index2.html?nounce=4314"]
>>> inside final response...
Final Response (2594.95 ms):
I have published the file `index2.html` to `https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index2.html?nounce=4314`.

-----------------------------
Total elapsed time: 2594.95


In [105]:
events = call_agent('redesign it using bento grid and save to index3.html')
pprint_events(events)

+++ inside the function call...
function, [args]:  save_text_file, {'file_text': '<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>AI Agent 2025 Research Plan - Bento Grid</title>\n    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">\n</head>\n<body class="bg-gray-100 font-sans">\n    <div class="container mx-auto p-8 grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4">\n\n        <!-- Main Title -->\n        <div class="col-span-full bg-white p-4 rounded-lg shadow-md">\n            <h1 class="text-3xl font-bold mb-4">AI Agent 2025 Research Plan</h1>\n            <p class="text-gray-700"><strong class="font-medium">Goal:</strong> To understand the landscape of AI Agents in 2025.</p>\n        </div>\n\n        <!-- Objectives -->\n        <div class="bg-white p-4 rounded-lg shadow-md">\n            <h2 class="text-xl font-semi

In [106]:
events = call_agent('publish that to /adk-101/demo-artifact-service-plan/')
pprint_events(events)

+++ inside the function call...
function, [args]:  publish_file, {'file_name': 'index3.html', 'path_prefix': '/adk-101/demo-artifact-service-plan/'}
loaded artifacts: <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content
--- inside the function call response...
Function Name: publish_file
Function Results ["File 'index3.html' published to https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index3.html?nounce=9615"]
>>> inside final response...
Final Response (2601.28 ms):
I have published the file `index3.html` to `https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index3.html?nounce=9615`.

-----------------------------
Total elapsed time: 2601.28


In [107]:
events = call_agent('change the color scheme to be sci-fi style, save it to index4.html and publish that to /adk-101/demo-artifact-service-plan/')
pprint_events(events)

+++ inside the function call...
function, [args]:  save_text_file, {'filename': 'index4.html', 'mime_type': 'text/html', 'file_text': '<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>AI Agent 2025 Research Plan - Sci-Fi</title>\n    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">\n</head>\n<body class="bg-gray-900 text-gray-100 font-sans">\n    <div class="container mx-auto p-8 grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4">\n\n        <!-- Main Title -->\n        <div class="col-span-full bg-gray-800 p-4 rounded-lg shadow-md border border-blue-500">\n            <h1 class="text-3xl font-bold mb-4 text-blue-300">AI Agent 2025 Research Plan</h1>\n            <p class="text-gray-300"><strong class="font-medium text-blue-200">Goal:</strong> To understand the landscape of AI Agents in 2025.</p>\n        </div>\n\n        <

+++ inside the function call...
function, [args]:  publish_file, {'file_name': 'index4.html', 'path_prefix': '/adk-101/demo-artifact-service-plan/'}
loaded artifacts: <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content
--- inside the function call response...
Function Name: publish_file
Function Results ["File 'index4.html' published to https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index4.html?nounce=8332"]
>>> inside final response...
Final Response (9314.321 ms):
I've created `index4.html` with a sci-fi color scheme and published it to `https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index4.html?nounce=8332`.

-----------------------------
Total elapsed time: 9314.321


In [108]:
events = call_agent('add some tailwind page load transition, save it to index5.html and publish that to /adk-101/demo-artifact-service-plan/')
pprint_events(events)

+++ inside the function call...
function, [args]:  save_text_file, {'file_text': '<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>AI Agent 2025 Research Plan - Sci-Fi with Transition</title>\n    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">\n    <style>\n        body {\n            opacity: 0;\n            transition: opacity 0.5s ease-in-out;\n        }\n        body.loaded {\n            opacity: 1;\n        }\n    </style>\n</head>\n<body class="bg-gray-900 text-gray-100 font-sans">\n    <div class="container mx-auto p-8 grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4">\n\n        <!-- Main Title -->\n        <div class="col-span-full bg-gray-800 p-4 rounded-lg shadow-md border border-blue-500">\n            <h1 class="text-3xl font-bold mb-4 text-blue-300">AI Agent 2025 Research Plan</h1>\n            <p class="te

+++ inside the function call...
function, [args]:  publish_file, {'file_name': 'index5.html', 'path_prefix': '/adk-101/demo-artifact-service-plan/'}
loaded artifacts: <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content
--- inside the function call response...
Function Name: publish_file
Function Results ["File 'index5.html' published to https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index5.html?nounce=7574"]
>>> inside final response...
Final Response (8997.642 ms):
I've added a page load transition to `index5.html` and published it to `https://storage.googleapis.com/hupili-genai-bb-public/adk-101/demo-artifact-service-plan/index5.html?nounce=7574`.

-----------------------------
Total elapsed time: 8997.642


In [109]:
events = call_agent('Yes, publish that to /adk-101/demo-artifact-service-plan/')
pprint_events(events)

>>> inside final response...
Final Response (1093.278 ms):
I already published `index5.html` to `/adk-101/demo-artifact-service-plan/` in the previous turn. Did you want me to publish it again?

-----------------------------
Total elapsed time: 1093.278
